<a href="https://colab.research.google.com/github/dangtrantanluc/FaceRecognition/blob/main/facerecognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install arcface

In [1]:
import insightface
from insightface.app import FaceAnalysis
from keras.models import Model
from keras.layers import Dense, GlobalMaxPooling2D, Input
from keras.optimizers import Adam
from keras.optimizers import RMSprop
from keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import InceptionResNetV2

# from tensorflow.keras.applications import InsightFaceResNet50

C:\Users\Harvey\AppData\Roaming\Python\Python312\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
C:\Users\Harvey\AppData\Roaming\Python\Python312\site-packages\albumentations\__init__.py:28: UserWarning: A new version of Albumentations is available: '2.0.7' (you have '2.0.6'). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()


In [ ]:
app = FaceAnalysis()
app.prepare(ctx_id=0)

if app.models:
  detector = app.models.get('retinaface_r50_v1', None)
  if detector:
    detector.nms = 0.4

download_path: /root/.insightface/models/buffalo_l


100%|██████████| 281857/281857 [00:10<00:00, 27908.96KB/s]
/usr/local/lib/python3.10/dist-packages/onnxruntime/capi/onnxruntime_inference_collection.py:115: UserWarning: Specified provider 'CUDAExecutionProvider' is not in available provider names.Available providers: 'AzureExecutionProvider, CPUExecutionProvider'
  warnings.warn(


Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /root/.insightface/models/buffalo_l/1k3d68.onnx landmark_3d_68 ['None', 3, 192, 192] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /root/.insightface/models/buffalo_l/2d106det.onnx landmark_2d_106 ['None', 3, 192, 192] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /root/.insightface/models/buffalo_l/det_10g.onnx detection [1, 3, '?', '?'] 127.5 128.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /root/.insightface/models/buffalo_l/genderage.onnx genderage ['None', 3, 96, 96] 0.0 1.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: /root/.insightface/models/buffalo_l/w600k_r50.onnx recognition ['None', 3, 112, 112] 127.5 127.5
set det-size: (640, 640)


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
base_model = InceptionResNetV2(weights="imagenet", include_top=False, input_shape=(224, 224, 3))
for layer in base_model.layers:
  layer.trainable = False
num_classes = 5

x = base_model.output
x = GlobalMaxPooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(num_classes, activation="softmax")(x)

model = Model(inputs=base_model.input, outputs=predictions)

model.compile(optimizer=Adam(learning_rate=0.001), loss="categorical_crossentropy", metrics=["accuracy"])
print(model.summary())



154664960/219055592 ━━━━━━━━━━━━━━━━━━━━ 4:36 4us/step

In [ ]:
train_dir = '/content/drive/MyDrive/DataSet/Train'
val_dir = '/content/drive/MyDrive/DataSet/Validate'

# Prepare data generators
train_datagen = ImageDataGenerator(
    rescale=1.0 / 255.0,
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    validation_split=0.2,
    fill_mode='nearest'

)
val_datagen = ImageDataGenerator(rescale=1.0 / 255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode="categorical")
validation_generator = train_datagen.flow_from_directory(
    val_dir  ,
    target_size=(224, 224),
    batch_size=32,
    class_mode="categorical")

label_map = train_generator.class_indices
label_map = {v: k for k, v in label_map.items()}

print("Label Map:", label_map)

Found 100 images belonging to 5 classes.
Found 15 images belonging to 5 classes.
Label Map: {0: 'Dang Tran Tan Luc', 1: 'Nguyen Thi Ngoc Diem', 2: 'Phung Khanh Duy', 3: 'Vo Nguyen Thanh Dieu', 4: 'Vo Thi Cam Tu'}


In [ ]:
checkpoint = ModelCheckpoint("/content/model.keras",
                             monitor="val_loss",
                             mode="min",
                             save_best_only = True,
                             verbose=1)

earlystop = EarlyStopping(monitor = 'val_loss',
                          min_delta = 0,
                          patience = 6,
                          verbose = 1,
                          restore_best_weights = True)

callbacks = [earlystop, checkpoint]
model.fit(
    train_generator,
    validation_data=validation_generator,
    epochs=20,
    callbacks = callbacks,
    steps_per_epoch=len(train_generator),
    validation_steps=len(validation_generator),
)


Epoch 1/20


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 6s/step - accuracy: 0.2902 - loss: 11.9002
Epoch 1: val_loss improved from inf to 26.13095, saving model to /content/model.keras
4/4 ━━━━━━━━━━━━━━━━━━━━ 81s 12s/step - accuracy: 0.2861 - loss: 13.2637 - val_accuracy: 0.2000 - val_loss: 26.1310
Epoch 2/20


/usr/lib/python3.10/contextlib.py:153: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 247ms/step - accuracy: 0.0000e+00 - loss: 0.0000e+00
Epoch 3/20


/usr/local/lib/python3.10/dist-packages/keras/src/callbacks/early_stopping.py:155: UserWarning: Early stopping conditioned on metric `val_loss` which is not available. Available metrics are: accuracy,loss
  current = self.get_monitor_value(logs)
/usr/local/lib/python3.10/dist-packages/keras/src/callbacks/model_checkpoint.py:206: UserWarning: Can save best model only with val_loss available, skipping.
  self._save_model(epoch=epoch, batch=None, logs=logs)


4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 7s/step - accuracy: 0.1956 - loss: 21.8459 
Epoch 3: val_loss improved from 26.13095 to 6.11575, saving model to /content/model.keras
4/4 ━━━━━━━━━━━━━━━━━━━━ 50s 10s/step - accuracy: 0.2005 - loss: 21.0406 - val_accuracy: 0.4000 - val_loss: 6.1158
Epoch 4/20
4/4 ━━━━━━━━━━━━━━━━━━━━ 2s 520ms/step - accuracy: 0.0000e+00 - loss: 0.0000e+00
Epoch 5/20
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 7s/step - accuracy: 0.4113 - loss: 5.8999
Epoch 5: val_loss did not improve from 6.11575
4/4 ━━━━━━━━━━━━━━━━━━━━ 38s 9s/step - accuracy: 0.3991 - loss: 6.2633 - val_accuracy: 0.2667 - val_loss: 9.1265
Epoch 6/20
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step - accuracy: 0.0000e+00 - loss: 0.0000e+00
Epoch 7/20
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 7s/step - accuracy: 0.2930 - loss: 7.2031
Epoch 7: val_loss improved from 6.11575 to 3.36460, saving model to /content/model.keras
4/4 ━━━━━━━━━━━━━━━━━━━━ 42s 10s/step - accuracy: 0.3004 - loss: 7.0294 - val_accuracy: 0.4667 - val_loss: 3.3646
Epoch 8/20
4/4 

In [ ]:
from tensorflow.keras.models import load_model
import cv2
import os
import numpy as np
from os import listdir
from os.path import isfile, join
from google.colab.patches import cv2_imshow


In [ ]:
classifier = load_model("/content/model.keras")

def draw_test(name, pred, im):
    face = label_map[pred[0]]
    BLACK = [0,0,0]
    expanded_image = cv2.copyMakeBorder(im, 80, 0, 0, 100 ,cv2.BORDER_CONSTANT,value=BLACK)
    cv2.putText(expanded_image, face, (20, 60) , cv2.FONT_HERSHEY_SIMPLEX,0.5, (0,0,255), 1)
    cv2_imshow( expanded_image)

def getRandomImage(path):
    """function loads a random images from a random folder in our test path """
    folders = list(filter(lambda x: os.path.isdir(os.path.join(path, x)), os.listdir(path)))
    random_directory = np.random.randint(0,len(folders))
    path_class = folders[random_directory]
    print("Class - " + path_class)
    file_path = path + path_class
    file_names = [f for f in listdir(file_path) if isfile(join(file_path, f))]
    random_file_index = np.random.randint(0,len(file_names))
    image_name = file_names[random_file_index]
    return cv2.imread(file_path+"/"+image_name)

for i in range(0,10):

    input_im = getRandomImage("/content/drive/MyDrive/DataSet/Validate/")
    input_original = input_im.copy()
    input_original = cv2.resize(input_original, None, fx=0.5, fy=0.5, interpolation = cv2.INTER_LINEAR)

    input_im = cv2.resize(input_im, (224, 224), interpolation = cv2.INTER_LINEAR)
    input_im = input_im / 255.
    input_im = input_im.reshape(1,224,224,3)

    res = np.argmax(classifier.predict(input_im, 1, verbose = 0), axis=1)

    draw_test("Prediction", res, input_original)
    cv2.waitKey(0)

cv2.destroyAllWindows()

Class - Nguyen Thi Ngoc Diem 


In [ ]:
import cv2
import numpy as np
from PIL import Image

label_map = {0: 'Dang Tran Tan Luc', 1: 'Nguyen Thi Ngoc Diem', 2: 'Phung Khanh Duy', 3: 'Vo Nguyen Thanh Dieu', 4: 'Vo Thi Cam Tu'}


def face_extractor(img):
    faces = face_cascade.detectMultiScale(img, 1.3, 5)
    if len(faces) == 0:  # Không tìm thấy khuôn mặt
        return None

    for (x, y, w, h) in faces:
        cv2.rectangle(img, (x, y), (x + w, y + h), (0, 255, 255), 2)
        cropped_face = img[y:y + h, x:x + w]
        return cropped_face

    return None

video_capture = cv2.VideoCapture(0)

if not video_capture.isOpened():
    print("Error: Cannot access the webcam.")
    exit()

while True:
    ret, frame = video_capture.read()
    if not ret:
        print("Failed to capture frame. Exiting...")
        break

    face = face_extractor(frame)
    if face is not None:
        try:
            face = cv2.resize(face, (224, 224))
            im = Image.fromarray(face, 'RGB')
            img_array = np.array(im)
            img_array = np.expand_dims(img_array, axis=0) / 255.0

            pred = classifier.predict(img_array)
            predicted_class = np.argmax(pred, axis=1)
            name = label_map[predicted_class[0]]

            cv2.putText(frame, name, (50, 50), cv2.FONT_HERSHEY_COMPLEX, 1, (0, 255, 0), 2)
        except Exception as e:
            print(f"Error during face processing: {e}")
    else:
        cv2.putText(frame, "No face detected", (50, 50), cv2.FONT_HERSHEY_COMPLEX, 1, (0, 255, 0), 2)

    if frame is not None and frame.size > 0:
        cv2.imshow('Video', frame)
    else:
        print("Empty frame detected. Skipping...")

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

video_capture.release()
cv2.destroyAllWindows()
